## Setup

In [ ]:
# Imports
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score
from datetime import timedelta
import random
import pickle
import matplotlib.pyplot as plt
import lightgbm as lgb

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
# Cambiamos el directorio para importar las funciones de func_aux
import sys
import os
from pathlib import Path
sys.path.append(str(Path(os.getcwd()).absolute().parent))

In [ ]:
# Importamos las funciones auxiliares
from src.funcs_aux import *

In [ ]:
# Importamos los datos pre-procesados
datos_unidos = pd.read_csv("../data/procesados/datos_unidos.csv")

In [ ]:
datos_unidos.columns

In [ ]:
cols_categoricas = ['SKU', 'STORE_ID', 'REGION',
       'CITY', 'STATE', 'STORE_TYPE',  'CATEGORY', 'GROUP', 'SUBGROUP', 'GROUP_TYPE',
       'PRICE_GROUP_ID', 'BRAND',
       "DAY_OF_WEEK", "WEEK", "DAY", "MONTH"]

In [ ]:
# Ordenamos y cambiamos los tipos de datos para optimizar memoria
datos_unidos.sort_values(by=["DATE", "STORE_ID", "SKU"], inplace=True)

for col in cols_categoricas:
    datos_unidos[col] = datos_unidos[col].astype("category")

datos_unidos["DATE"] = datos_unidos["DATE"].astype("datetime64[ns]")

In [ ]:
# Completamos el dataset con todas las combinaciones de SKU, STORE_ID y DATE, de manera que el modelo pueda ver los días sin ventas
datos_unidos = completar_dataset(datos_unidos)

# Completamos las columnas faltantes
datos_unidos["YEAR"] = datos_unidos["DATE"].dt.year
datos_unidos["MONTH"] = datos_unidos["DATE"].dt.month
datos_unidos["DAY"] = datos_unidos["DATE"].dt.day
datos_unidos["DAY_OF_WEEK"] = datos_unidos["DATE"].dt.day_name()
datos_unidos["WEEK"] = datos_unidos["DATE"].dt.isocalendar().week
datos_unidos["CLOSEDATE"] = datos_unidos["CLOSEDATE"].astype("datetime64[ns]")
datos_unidos["OPENDATE"] = datos_unidos["OPENDATE"].astype("datetime64[ns]")
datos_unidos["MONTH_CLOSE"] = datos_unidos["CLOSEDATE"].dt.month
datos_unidos["YEAR_CLOSE"] = datos_unidos["CLOSEDATE"].dt.year
datos_unidos["MONTH_OPEN"] = datos_unidos["OPENDATE"].dt.month
datos_unidos["YEAR_OPEN"] = datos_unidos["OPENDATE"].dt.year

In [ ]:
for col in cols_categoricas:
    datos_unidos[col] = datos_unidos[col].astype("category")

In [ ]:
# Nos aseguramos que no haya valores nulos
datos_unidos.isna().sum()

## Feature aggregation

Haremos el promedio y desviación estándar de las ventas por subgrupo y por categoría, de manera que el modelo pueda entender mejor los cambios de ventas por épocas del año de grupos más grandes de productos.

In [ ]:
rolling_sales(datos_unidos, group="SUBGROUP", windows=[7, 30, 90], std=False)
rolling_sales(datos_unidos, group="SKU", windows=[7, 30, 90], std=False)
rolling_sales(datos_unidos, group="STORE_ID", windows=[30, 90], std=False)

Haremos lo mismo, pero por los cambios porcentuales del precio por SKU y por SUBGROUP, de manera que el modelo pueda entender la estacionalidad también de los precios

In [ ]:
datos_unidos = rolling_price_pct(datos_unidos=datos_unidos, group="SKU", windows=[30,90], std=False)
datos_unidos = rolling_price_pct(datos_unidos=datos_unidos, group="SUBGROUP", windows=[30,90], std=False)

## Walk-forward validation LightGBM

In [ ]:
cols = list(datos_unidos.columns)
features = [col for col in cols if col not in ["DATE", "TOTAL_SALES",
                                               'INITIAL_TICKET_PRICE', 'BASE_PRICE', "COSTOS",
                                               "OPENDATE", "CLOSEDATE", "QUANTITY", "STORE_SUBGROUP_DATE_ID"] ]

print(features)

A continuación, hacemos un walk-forward validation con LightGBM: entrenamos primero el modelo con 365 días de datos, predecimos las ventas de los próximos 7 días, sumamos 30 días de datos y repetimos el proceso, guardando la métrica R2 calculada de los datos de test.

Además, para agilizar el proceso, utiliza 7 días de los datos de entrenamiento como data validation, de manera que pueda detenerse si pasan muchos rounds sin mejorar la performance sobre estos datos.

In [ ]:
results_lgb = walk_forward_lightgbm(
    df=datos_unidos,
    features=features,
    target_col="TOTAL_SALES",
    date_col="DATE",
    categorical_cols=cols_categoricas,
    train_days=365,
    step_days=180,
    forecast_days=14,
    params={
        "objective": "regression",
        "metric": "rmse",
        "verbosity": 2,
        "learning_rate": 0.01,
        "num_leaves": 1500,
        "max_depth": 50,
        "min_data_in_leaf": 1000,
        "feature_fraction": 1,
        "bagging_fraction": 1,
        "bagging_freq": 0,
        "early_stopping_round": 50,
        "num_boost_round":1000
    }
)

In [ ]:
results_lgb.mean()

Vale aclarar que el fine-tuning de los hiperparámetros lo hicimos manualmente, debido a que el anterior walk-forward demora bastante tiempo.

## Deploy

En esta sección, entrenaremos el modelo con todos los datos disponibles y utilizaremos los hiperparámetros que mejor performance tuiveron en el walk-forward validation. 

Después, contruiremos un dataframe template con todas las combinaciones de DATE, SKU y STORE_ID de la próxima semana. 

Por último, valiéndonos de la librería de Optuna, usaremos optimización bayesiana para encontrar la mejor configuración de precios que maximiza las ganancias de la próxima semana, según las predicciones de nuestro modelo.

### Training

In [ ]:
cols = list(datos_unidos.columns)
features = [col for col in cols if col not in ["DATE", "TOTAL_SALES",
                                               'INITIAL_TICKET_PRICE', 'BASE_PRICE', "COSTOS",
                                               "OPENDATE", "CLOSEDATE", "QUANTITY", "STORE_SUBGROUP_DATE_ID"] ]

print(features)

In [ ]:
params={
        "objective": "regression",
        "metric": "rmse",
        "verbosity": 2,
        "learning_rate": 0.01,
        "num_leaves": 100,
        "max_depth": 50,
        "min_data_in_leaf": 100,
        "feature_fraction": 1,
        "bagging_fraction": 1,
        "bagging_freq": 0,
        "num_boost_round":100
    }

In [ ]:
data_train = lgb.Dataset(datos_unidos[features], datos_unidos["TOTAL_SALES"], categorical_feature=cols_categoricas)

In [ ]:
model_lgb = lgb.train(params, data_train)

### Template dataframe

Primero, creamos un dataframe base con todos los productos y tiendas para los 7 días, de manera que el modelo pueda predecir las ventas de cada combinación

In [ ]:
columnas_extraidas = ['SKU', 'STORE_ID', 'REGION',
       'CITY', 'STATE', 'STORE_TYPE', 'CATEGORY', 'GROUP', 'SUBGROUP', 'GROUP_TYPE',
       'PRICE_GROUP_ID', 'BRAND', 'YEAR_OPEN', 'YEAR_CLOSE', 'MONTH_OPEN', 'MONTH_CLOSE']

In [ ]:
template = crear_template(datos_unidos, columnas_extraidas=columnas_extraidas, cols_categoricas=cols_categoricas)

In [ ]:
len(template)

In [ ]:
# Para las columnas con categorias que pueden no aparecer en template
for col in ["STORE_ID", "MONTH", "WEEK"]:
    type_stores = pd.api.types.CategoricalDtype(categories=datos_unidos[col].unique())
    template[col] = template[col].astype(type_stores)

In [ ]:
template = template.sort_values(by=["DATE", "STORE_ID", "SKU"])

In [ ]:
# Para los rolling features, simplificamos el proceso y tomamos los últimos días de los datos de entrenamiento
template = rolling_sales_template(df=datos_unidos, template=template, group= "SUBGROUP", windows=[7, 30, 90], std=False)
template = rolling_sales_template(df=datos_unidos, template=template, group= "SKU", windows=[7, 30, 90], std=False)
template = rolling_sales_template(df=datos_unidos, template=template, group= "STORE_ID", windows=[30, 90], std=False)
template = rolling_price_template(datos_unidos, template, "SUBGROUP", windows=[30,90], std=False)
template = rolling_price_template(datos_unidos, template, "SKU", windows=[30,90], std=False)

In [ ]:
template.columns

### Optimizacion de precios

Para optimizar los precios, ya calculamos una grid de precios posibles por SKU en el analisis de datos, quitando aquellos precios que distaban demasiado de la distribución normal de precios, y quedándonos con el máximo y mínimo de las transacciones del último mes.

Aún así, también probamos utilizando el máximo y mínimo histórico del producto, agregando además posibles descuentos, pero los resultados fueron peores.

In [ ]:
with open("resultados_optimizacion/price_grid_normalizado.pkl", "rb") as f:
    price_grid = pickle.load(f)

Ahora, utilizando las predicciones del modelo, intentamos encontrar qué configuración de precios maximiza TOTAL_SALES. 

Para simplificar la posterior implementación de la estrategia de precios, decidimos que cada precio de SKU se mantenga durante toda la semana en todas las tiendas

In [ ]:
df_best, y_best, mejor_sales, mejor_gain, precios_finales, study = optimizacion_precios_optuna(template=template, model=model_lgb, price_grid=price_grid,
                                                                                      features=features, target="TOTAL_SALES", file_name="mejor_config_optuna",
                                                                                      n_trials=10, seed=42)

In [ ]:
# TOTAL_SALES de la mejor configuración encontrada
print(mejor_sales)

In [ ]:
# Ganancia total de la mejor configuración encontrada (TOTAL_SALES - COSTOS)
print(mejor_gain)

También probamos maximizar la ganancia en lugar de TOTAL_SALES (se puede pasar como parámetro target="GAIN"), pero los resultados fueron los mismos.

Además, intentamos optimizar cambiando los precios por región (es decir, las tiendas de diferentes regiones pueden tener distintas configuraciones de precios), aunque otra vez vimos los mismos resultados.

In [ ]:
df_best, y_best, mejor_sales, mejor_gain, precios_finales, study = optimizacion_precios_region_optuna(template=template, model=model_lgb, price_grid=price_grid,
                                                                                      features=features, target="TOTAL_SALES", file_name="mejor_config_optuna",
                                                                                      n_trials=1000, seed=42)

### Guardar resultado

Finalmente, guardamos las predicciones de la próxima semana con los precios optimizados con el formato de Kaggle. 

Tuvimos que agregar manualmente el subgrupo "Basketball" siempre con TOTAL_SALES = 0, dado que en los datos disponibles nunca se hizo una transacción de alguno de sus productos, pero igualmente aparecía en el catágolo.

In [ ]:
df_result = crear_csv_kaggle(df_best, dummy_subgroup="Basketball")

In [ ]:
df_result.to_csv("resultados_optimizacion/mejor_df_completo.csv", index=False)

## Interpretación del modelo

Gracias a que el modelo utilizado está basado en árboles de decisión, es muy interpretable y podemos ver qué features fueron más relevantes

In [ ]:
# Extraer importancias
importancia = model_lgb.feature_importance(importance_type="gain")
features = model_lgb.feature_name()
df_importancia = pd.DataFrame({
    "feature": features,
    "importance": importancia / importancia.sum() * 100 
}).sort_values("importance", ascending=False).head(20)

# Graficar en horizontal
ax = df_importancia.plot.barh(x="feature", y="importance", legend=False, figsize=(8,6))
plt.xlabel("Importancia (%)")

# Agregar porcentajes al lado
for i, v in enumerate(df_importancia["importance"]):
    ax.text(v + 0.5, i, f"{v:.1f}%", va="center")

plt.title("Importancia de los features")
plt.grid()
plt.savefig("importancia_features.png", bbox_inches='tight', dpi=300)
plt.show()